In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim


In [2]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

csv_file = "./eda_features.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=480, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)


In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 6
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 4)
        self.fc2 = nn.Linear(4, 2)
        self.fc3 = nn.Linear(2, self.out_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        output = self.fc3(a2)  # Pas besoin de log_softmax pour la régression
        return output

In [16]:
model = MLP()
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(model.parameters(), lr=0.0001)

for epoch in range(10):
    running_loss = 0.0
    ancienRunning=-1
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
        if (i + 1) % 200 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,   200] loss: 8602.641
[1,   400] loss: 0.065
[1,   600] loss: 0.062
[1,   800] loss: 0.059
[1,  1000] loss: 0.056
[1,  1200] loss: 0.054
[1,  1400] loss: 0.052
[2,   200] loss: 0.049
[2,   400] loss: 0.047
[2,   600] loss: 0.046
[2,   800] loss: 0.044
[2,  1000] loss: 0.043
[2,  1200] loss: 0.041
[2,  1400] loss: 0.040
[3,   200] loss: 0.038
[3,   400] loss: 0.038
[3,   600] loss: 0.037
[3,   800] loss: 0.036
[3,  1000] loss: 0.035
[3,  1200] loss: 0.034
[3,  1400] loss: 0.033
[4,   200] loss: 0.033
[4,   400] loss: 0.032
[4,   600] loss: 0.031
[4,   800] loss: 0.031
[4,  1000] loss: 0.030
[4,  1200] loss: 0.030
[4,  1400] loss: 0.030
[5,   200] loss: 0.029
[5,   400] loss: 0.029
[5,   600] loss: 0.028
[5,   800] loss: 0.028
[5,  1000] loss: 0.028
[5,  1200] loss: 0.028
[5,  1400] loss: 0.027
[6,   200] loss: 0.027
[6,   400] loss: 0.027
[6,   600] loss: 0.027
[6,   800] loss: 0.027
[6,  1000] loss: 0.026
[6,  1200] loss: 0.026
[6,  1400] loss: 0.026
[7,   200] loss: 0.026
[7,   40

In [17]:
def test_model(model, test_loader, criterion):
    model.eval()  # Mettre le modèle en mode évaluation
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver le calcul des gradients pendant l'évaluation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Convertir les étiquettes en type float
            test_loss += loss.item()
            # Calcul de la précision
            predicted = torch.round(outputs).squeeze()  # Arrondir les prédictions
            #print(outputs ,"     rechercher : " , labels) 
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Afficher les résultats
    avg_loss = test_loss / len(test_loader)
    accuracy = correct / total
    print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(avg_loss, accuracy * 100))

# Utiliser la fonction test_model pour évaluer le modèle
test_model(model, test_loader, criterion)


Test Loss: 0.2474, Accuracy: 56.50%


In [7]:
# Supposons que 'model' soit votre modèle PyTorch
# Vous pouvez afficher les poids des neurones comme ceci :
for name, param in model.named_parameters():
    if 'weight' in name:
        print(name, param.data)

fc1.weight tensor([[-1.4461e+05, -3.1620e-01, -5.0942e-01, -6.7689e-01, -2.0451e+00,
         -4.2608e-01],
        [-4.4345e+04,  1.5147e-01, -3.1170e-01, -2.6614e-01, -7.6081e-01,
          9.8790e-02],
        [-1.7681e-01,  3.4868e-01, -3.4122e-01,  3.0015e-02,  2.2122e-01,
          9.9120e-02],
        [-4.0587e-02,  2.4573e-02, -3.3553e-01,  2.5931e-01,  2.5914e-01,
         -2.8293e-01]])
fc2.weight tensor([[ 2.4587e-01, -4.4254e-01,  3.9038e-01,  9.6435e-02],
        [-1.2433e+05, -1.4219e+05,  6.0256e-02,  9.1557e-02]])
fc3.weight tensor([[-2.7233e-01, -2.5801e+05]])
